In [44]:
# Initial imports
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

%matplotlib inline

In [45]:
# Import Verlander dataset 

verlander_df = pd.read_csv(Path("../resources/verlander_update.csv"),
                            index_col='game_date',
                            parse_dates=True,
                            infer_datetime_format=True)

display(verlander_df.head())

,pitch_type,player_name,batter,pitcher,events,description,zone,des,stand,p_throws,...,on_1b,outs_when_up,inning,at_bat_number,pitch_number,pitch_name,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment
game_date,,,,,,,,,,,,,,,,,,,,,
2022-09-16,SL,"Verlander, Justin",669127,434378,strikeout,swinging_strike,9,Shea Langeliers strikes out swinging.,R,R,...,NaN,2,5,35,5,Slider,0,2,Infield shift,Standard
2022-09-16,FF,"Verlander, Justin",669127,434378,NaN,foul,3,Shea Langeliers strikes out swinging.,R,R,...,NaN,2,5,35,4,4-Seam Fastball,0,2,Infield shift,Standard
2022-09-16,SL,"Verlander, Justin",669127,434378,NaN,called_strike,13,Shea Langeliers strikes out swinging.,R,R,...,NaN,2,5,35,3,Slider,0,2,Infield shift,Standard
2022-09-16,FF,"Verlander, Justin",669127,434378,NaN,foul,1,Shea Langeliers strikes out swinging.,R,R,...,NaN,2,5,35,2,4-Seam Fastball,0,2,Infield shift,Standard
2022-09-16,SL,"Verlander, Justin",669127,434378,NaN,ball,8,Shea Langeliers strikes out swinging.,R,R,...,NaN,2,5,35,1,Slider,0,2,Infield shift,Standard


In [46]:
# Clean dataset 

verlander_df = verlander_df.drop(columns=['des', 
'pitch_name', 
'at_bat_number', 
'inning', 
'zone', 
'type', 
'player_name', 
'batter', 
'pitcher', 
'events',
'bb_type',
'hit_location'])

display(verlander_df.head())

,pitch_type,description,stand,p_throws,home_team,away_team,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,pitch_number,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment
game_date,,,,,,,,,,,,,,,,,
2022-09-16,SL,swinging_strike,R,R,HOU,OAK,1,2,NaN,NaN,NaN,2,5,0,2,Infield shift,Standard
2022-09-16,FF,foul,R,R,HOU,OAK,1,2,NaN,NaN,NaN,2,4,0,2,Infield shift,Standard
2022-09-16,SL,called_strike,R,R,HOU,OAK,1,1,NaN,NaN,NaN,2,3,0,2,Infield shift,Standard
2022-09-16,FF,foul,R,R,HOU,OAK,1,0,NaN,NaN,NaN,2,2,0,2,Infield shift,Standard
2022-09-16,SL,ball,R,R,HOU,OAK,0,0,NaN,NaN,NaN,2,1,0,2,Infield shift,Standard


In [47]:
# Split into X and y 

X = verlander_df.drop(columns='pitch_type')
y= verlander_df['pitch_type']

In [70]:
# Use get_dummies to encode categorical variables 

X = pd.get_dummies(X)



# Encode on-base with 1s and 0s 

for label, content in verlander_df.iterrows():
    if content == "on_3b" or content == "on_2b" or content == "on_1b":
        print('here!')
        print(content)
        if content != np.nan:
            content = 1
        else:
            continue
            print("I'm here now")
    else: 
        continue
print("phew finally finished")



#display(X.head())
#X['on_1b'].value_counts()

phew finally finished


In [ ]:
# Initialize the Deep Learning Neural Network model

nn_v0 = Sequential()

In [ ]:
# Design the network architecture 

nn_v0.add(Dense())